# CommaSeparatedListOutputParser
- LLM 출력을 쉼표로 구분된 리스트로 반환하고 싶을 때 사용한다.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
print(os.environ["MODEL_ID"])

meta-llama/Meta-Llama-3-8B-Instruct


In [8]:
from langchain_community.chat_models.huggingface import ChatHuggingFace
from langchain_community.llms import HuggingFaceEndpoint

llm = HuggingFaceEndpoint(
    repo_id=os.environ["MODEL_ID"], 
    # max_new_tokens=1024,
    temperature=0.1,
    huggingfacehub_api_token=os.environ["HF_API_KEY"],
)
model = ChatHuggingFace(llm=llm)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/dudaji/.cache/huggingface/token
Login successful


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate

output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="List five {subject}\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

chain = prompt | model | output_parser
print(format_instructions)

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [10]:
chain.invoke({"subject": "Attractions of South Korea"})

['Here are five attractions of South Korea:\n\nGyeongbokgung Palace',
 'Bukchon Hanok Village',
 'Myeong-dong Shopping District',
 'Nami Island',
 'Jeju Island']

`chain.stream`을 사용하여 스트림에 대해 반복적으로 parsing 되도록 할 수 있다

In [11]:
for chunk in chain.stream({"subject": "Attractions of South Korea"}):
    print(chunk)

['Here are five attractions of South Korea:\n\nGyeongbokgung Palace']
['Bukchon Hanok Village']
['Myeong-dong Shopping District']
['Nami Island']
['Demilitarized Zone (DMZ)<|eot_id|>']
